In [1]:
import os
from collections import defaultdict
import sys
import time
import json
import psutil
import argparse
import operator
import pandas as pd
sys.path.append("/nykaa/api")
from pas.v2.utils import Utils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [ ]:
nykaadb = DiscUtils.nykaaMysqlConnection()
cursor = nykaadb.cursor()

In [ ]:
# Construct a list/map of L1 categories from the start so that the orders query is more human-readable
# There are only ~15 L1 categories
l1_categories_map = {}

l1_categories_query = "SELECT entity_id, value " \
                      "FROM catalog_category_entity_varchar " \
                      "WHERE entity_id IN " \
                      "(" \
                            "SELECT cce.entity_id " \
                            "FROM catalog_category_entity cce " \
                            "LEFT JOIN catalog_category_entity_int ccei ON ccei.entity_id = cce.entity_id " \
                            "WHERE cce.level = 2 AND ccei.attribute_id = 498 AND ccei.value = 1 " \
                      ") " \
                      "AND attribute_id = 31;"

cursor.execute(l1_categories_query)

for row in cursor.fetchall():
    l1_categories_map[row[0]] = row[1]

# the string that will be directly inserted/substituted into the orders query
l1_categories_list_str = ", ".join([str(x) for x in l1_categories_map])

# Function for extracting out data from big queries

In [ ]:
def extract_data(query):
    cursor.execute(query)
    rows = []
    BATCH_SIZE = 10000

    while True:
        batch_empty = True
        for row in cursor.fetchmany(BATCH_SIZE):
            batch_empty = False
            rows.append(row)
        if batch_empty:
            break
            
    return rows

Initializing dataframe for storing all the data and then dumping it into csv

In [2]:
df = pd.DataFrame()

# Current Orders Table

In [ ]:
orders_query = "SELECT sfoi.order_id, sfoi.product_id, sfoi.product_type, ccpi.category_id, eaov.value " \
                       "FROM sales_flat_order_item sfoi FORCE INDEX (IDX_SALES_FLAT_ORDER_ITEM_ORDER_ID) " \
                       "LEFT JOIN catalog_product_entity_varchar cpev ON cpev.entity_id = sfoi.product_id AND cpev.attribute_id = 668 " \
                       "LEFT JOIN eav_attribute_option_value eaov ON eaov.option_id = cpev.value AND eaov.store_id = 0 " \
                       "LEFT JOIN catalog_category_product_index ccpi ON ccpi.product_id = sfoi.product_id " \
                       "WHERE (sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND ccpi.category_id IN (" + l1_categories_list_str + ") ); "

In [ ]:
rows = extract_data(orders_query)
df = df.append(rows)
len(rows), len(df)

# Archive Orders Table

In [ ]:
archived_orders_query = "SELECT sfoi.order_id, sfoi.product_id, sfoi.product_type, ccpi.category_id, eaov.value " \
                       "FROM sales_flat_order_item_archive sfoi " \
                       "LEFT JOIN catalog_product_entity_varchar cpev ON cpev.entity_id = sfoi.product_id AND cpev.attribute_id = 668 " \
                       "LEFT JOIN eav_attribute_option_value eaov ON eaov.option_id = cpev.value AND eaov.store_id = 0 " \
                       "LEFT JOIN catalog_category_product_index ccpi ON ccpi.product_id = sfoi.product_id " \
                       "WHERE (sfoi.order_id <> 0 AND sfoi.mrp > 1 AND sfoi.parent_item_id IS NULL AND ccpi.category_id IN (" + l1_categories_list_str + ") );"

In [ ]:
rows = extract_data(archived_orders_query)
df = df.append(rows)
len(rows), len(df)

In [ ]:
df.head()

In [ ]:
df.columns = ['order_id', 'product_id', 'product_type', 'category_id', 'brand']

In [ ]:
df.to_csv('fbt_orders_data_with_archived_combined.csv', encoding='utf-8', index=False)

# FBT data without categories

In [3]:
orders_query = "SELECT order_id, product_id FROM sales_flat_order_item WHERE order_id <> 0 AND mrp > 1; "

In [4]:
rows = DiscUtils.fetchResultsInBatch(DiscUtils.nykaaMysqlConnection(), orders_query, 10000)
df = df.append(rows)
len(rows), len(df)

(49192354, 49192354)

In [5]:
orders_archive_query = "SELECT order_id, product_id FROM sales_flat_order_item_archive WHERE order_id <> 0 AND mrp > 1; "

In [6]:
rows = DiscUtils.fetchResultsInBatch(DiscUtils.nykaaMysqlConnection(), orders_archive_query, 10000)
df = df.append(rows)
len(rows), len(df)

(8530541, 57722895)

In [7]:
df.head()

,0,1
0,720719,18516
1,720719,18514
2,720719,12125
3,720719,12124
4,768913,6048


In [8]:
df.columns = ['order_id', 'product_id']

In [9]:
df.head()

,order_id,product_id
0,720719,18516
1,720719,18514
2,720719,12125
3,720719,12124
4,768913,6048


In [10]:
query = "select child_id, parent_id from catalog_product_relation"

rows = DiscUtils.fetchResultsInBatch(DiscUtils.nykaaMysqlConnection(), query, 50000)
child_2_parent = {row[0]: row[1] for row in rows}

In [11]:
with open("child_product_2_parent.json", "r+") as f:
    child_2_parent.update({int(key): int(value) for key, value in json.load(f).items()})

In [12]:
df['product_count'] = 1

In [13]:
df.head()

,order_id,product_id,product_count
0,720719,18516,1
1,720719,18514,1
2,720719,12125,1
3,720719,12124,1
4,768913,6048,1


# Parallelizing DataFrame operations

In [14]:
from multiprocessing import Pool
import numpy as np

def parallelize_dataframe(df, func, num_partitions, num_cores):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [15]:
def convert_2_parent_product(data):
    data['product_id'] = data.apply(lambda row: child_2_parent.get(row['product_id'], row['product_id']), axis=1)
    return data

df = parallelize_dataframe(df, convert_2_parent_product, 40, 16)

In [16]:
df = df.groupby(['order_id', 'product_id']).agg({'product_count': 'count'}).reset_index()

In [17]:
df.head()

,order_id,product_id,product_count
0,386449,885,1
1,386449,16985,2
2,386449,19632,1
3,386449,19637,1
4,386450,3062,2


In [18]:
len(df)

37498790

In [19]:
df = df.drop(['product_count'], axis=1)

In [20]:
df.head()

,order_id,product_id
0,386449,885
1,386449,16985
2,386449,19632
3,386449,19637
4,386450,3062


In [21]:
df.to_csv('fbt_without_categories_6_july.csv', encoding='utf-8', index=False)